In [47]:
from collections import defaultdict, Counter
from itertools import product
from time import clock
import sys
import ast

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy.sparse as sps
from scipy.linalg import pinv
from scipy.spatial import distance 

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
from sklearn.cluster import KMeans as kmeans
from sklearn.mixture import GaussianMixture as GMM
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import adjusted_mutual_info_score as ami, accuracy_score as accuracy, pairwise_distances

# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC

In [45]:
# Helpers

def cluster_acc(Y, clusterLabels):
    assert (Y.shape == clusterLabels.shape)
    pred = np.empty_like(Y)
    for label in set(clusterLabels):
        mask = clusterLabels == label
        sub = Y[mask]
        target = Counter(sub).most_common(1)[0][0]
        pred[mask] = target
#    assert max(pred) == max(Y)
#    assert min(pred) == min(Y)    
    return accuracy(Y, pred)


class myGMM(GMM):
    def transform(self, X):
        return self.predict_proba(X)
        
        
def pairwiseDistCorr(X1, X2):
    assert X1.shape[0] == X2.shape[0]
    
    d1 = pairwise_distances(X1)
    d2 = pairwise_distances(X2)
    return np.corrcoef(d1.ravel(), d2.ravel())[0,1]

    
def aveMI(X, Y):    
    MI = MIC(X, Y) 
    return np.nanmean(MI)
    
  
def reconstructionError(projections, X):
    W = projections.components_
    if sps.issparse(W):
        W = W.todense()
    p = pinv(W)
    reconstructed = ((p@W)@(X.T)).T # Unproject projected data
    errors = np.square(X-reconstructed)
    return np.nanmean(errors)
    
# http://datascience.stackexchange.com/questions/6683/feature-selection-using-feature-importances-in-random-forests-with-scikit-learn          
class ImportanceSelect(BaseEstimator, TransformerMixin):
    def __init__(self, model, n=1):
         self.model = model
         self.n = n
    def fit(self, *args, **kwargs):
         self.model.fit(*args, **kwargs)
         return self
    def transform(self, X):
         return X[:,self.model.feature_importances_.argsort()[::-1][:self.n]]
                  
#http://stats.stackexchange.com/questions/90769/using-bic-to-estimate-the-number-of-k-in-kmeans    
def compute_bic(kmeans, X):
    """
    Computes the BIC metric for a given clusters
    Parameters:
    -----------------------------------------
    kmeans:  List of clustering object from scikit learn
    X     :  multidimension np array of data points
    Returns:
    -----------------------------------------
    BIC value
    """
    # assign centers and labels
    centers = [kmeans.cluster_centers_]
    labels  = kmeans.labels_
    #number of clusters
    m = kmeans.n_clusters
    # size of the clusters
    n = np.bincount(labels)
    #size of data set
    N, d = X.shape

    #compute variance for all clusters beforehand
    cl_var = (1.0 / (N - m) / d) * sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 'euclidean')**2) for i in range(m)])

    const_term = 0.5 * m * np.log(N) * (d+1)

    BIC = np.sum([n[i] * np.log(n[i]) -
               n[i] * np.log(N) -
             ((n[i] * d) / 2) * np.log(2*np.pi*cl_var) -
             ((n[i] - 1) * d/ 2) for i in range(m)]) - const_term

    return(BIC)

def plot_metric_vs_comp(df, problem_name, multiple_runs,
                        title, ylabel):

    plt.close()
    plt.figure()
    plt.title(title)
    plt.xlabel("N Components")
    plt.ylabel(ylabel)
    plt.grid()
    plt.tight_layout()

    ax = plt.gca()

    x_points = df.index.values
    y_points = df[1]
    if multiple_runs:
        y_points = np.mean(df.iloc[:, 1: -1], axis=1)
        y_std = np.std(df.iloc[:, 1: -1], axis=1)
        plt.plot(x_points, y_points, 'o-', linewidth=1, markersize=2,
                 label=ylabel)
        plt.fill_between(x_points, y_points - y_std,
                         y_points + y_std, alpha=0.2)
    else:
        plt.plot(x_points, y_points, 'o-', linewidth=1, markersize=2,
                 label=ylabel)

    plt.legend(loc="best")

    return plt

def run_plot_metric_vs_comp(out, dataset, metric, problem, title):
    file = out + '{}_{}.csv'.format(dataset, metric)
    df = pd.read_csv(file, header=None).dropna().set_index(0)
    multiple_runs = True if problem == 'RP' else False
    p = plot_metric_vs_comp(df=df, problem_name=problem, multiple_runs=multiple_runs,
                            title=title, ylabel=metric)

    p.savefig('{}/{}_{}.png'.format(out, dataset, metric),
                                    format='png', bbox_inches='tight', dpi=150)

def plot_complexity_curve(title, param, problem, classifier, dataset, ylim=None):
    plt.figure()
    plt.title(title)
    
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel(param)
    plt.ylabel("Score")

    grid_search = pd.read_csv('./{}/{}_cv_results.csv'.format(problem, dataset))
    best_columns = ['mean_train_score', 'mean_test_score', 'mean_fit_time', 'mean_score_time', 'params']
    best = pd.DataFrame(grid_search[best_columns].loc[grid_search.mean_test_score.idxmax()]).T.reset_index()
    best.to_csv('./{}/{}_best.csv'.format(problem, dataset))
    best_params = ast.literal_eval(best.loc[0, 'params'])

    best_params.pop('{}__{}'.format(classifier, param))
    for _param, value in best_params.items():
        if isinstance(value, tuple):
            grid_search['param_'+_param] = grid_search['param_'+_param].apply(ast.literal_eval)
        grid_search = grid_search.loc[grid_search['param_'+_param] == value]

    df = grid_search[['param_{}__{}'.format(classifier, param), 
                      'mean_test_score', 
                      'std_test_score', 
                      'mean_train_score', 
                      'std_train_score']].sort_values(by='param_{}__{}'.format(classifier, param))
    param_values = df['param_{}__{}'.format(classifier, param)]
    train_scores_mean = df['mean_train_score']
    train_scores_std = df['std_train_score']
    test_scores_mean = df['mean_test_score']
    test_scores_std = df['std_test_score']
    plt.grid()

    plt.fill_between(param_values, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2)
    plt.fill_between(param_values, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2)
    plt.plot(param_values, train_scores_mean, 'o-', linewidth=1, markersize=4,
             label="Train")
    plt.plot(param_values, test_scores_mean, 'o-', linewidth=1, markersize=4,
             label="Test")

    plt.legend(loc="best")
    return plt

def make_complexity_curve(clf_name, dataset, param, problem):    
    print('Making complexity curve for', dataset)
    plt = plot_complexity_curve('Complexity Curve: {} - {} - {}'.format(clf_name, dataset, param),
                                param,
                                problem,
                                clf_name,
                                dataset,
                                ylim=None)

    plt.savefig('./{}/{}_CC_{}.png'.format(problem, dataset, param), format='png', dpi=150)
    plt.close()
    return

In [5]:
# Data load and preprocessing

# Mushroom dataset
mushroom = pd.read_csv('mushroom.txt', header=None)
print('Dataset shape: ' + str(mushroom.shape))
mushroom.columns = ['type', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color','population', 'habitat']
mushroom['type_label'] = mushroom['type'].astype('category')
print('Mushroom types: ', str(mushroom['type_label'].cat.categories))
print('Labels balance: \n', mushroom['type_label'].value_counts()/mushroom['type_label'].size)
# Code 1 is for poisonous mushrooms

mushroom['type'] = mushroom['type_label'].cat.codes

# There are no null values, missing is represented as ? only for stalk root
print('Missing values for stalk root: '+ str(sum(mushroom['stalk-root']=='?')))

# We remove feature stalk-root due to high number of missing values
mushroom = pd.get_dummies(mushroom.drop(columns=['type_label','stalk-root']))
mushroom = mushroom.astype(float)
mushroom.type = mushroom.type.astype(int)

Dataset shape: (8124, 23)
Mushroom types:  Index(['e', 'p'], dtype='object')
Labels balance: 
 e    0.517971
p    0.482029
Name: type_label, dtype: float64
Missing values for stalk root: 2480


In [9]:
# Wine dataset

wine = pd.read_csv('winequality-white.csv', sep = ';')
print('Dataset shape: ', str(wine.shape))

# There are no null values in the dataset

# Creation of class for good vs bad wines (good when greater or equal than 7)
wine['quality_label'] = 'bad'
wine.loc[wine['quality']>=7, 'quality_label'] = 'good'
wine['quality_label'] = wine['quality_label'].astype('category')

print('Quality values: ', str(wine['quality_label'].cat.categories))
print('Labels balance: \n', wine['quality_label'].value_counts()/wine['quality_label'].size)

wine['quality_int'] = wine['quality']
wine['quality'] = wine['quality_label'].cat.codes

wine = wine.drop(['quality_label', 'quality_int'], axis=1)
wine = wine.astype(float)
wine.quality = wine.quality.astype(int)

Dataset shape:  (4898, 12)
Quality values:  Index(['bad', 'good'], dtype='object')
Labels balance: 
 bad     0.783585
good    0.216415
Name: quality_label, dtype: float64


In [29]:
# Load Data       
mushroomX = mushroom.drop('type', axis=1).values
mushroomY = mushroom['type'].values

wineX = wine.drop('quality', axis=1).values
wineY = wine['quality'].values

mushroomX = StandardScaler().fit_transform(mushroomX)
wineX = StandardScaler().fit_transform(wineX)

mushroom_trgX, mushroom_tstX, mushroom_trgY, mushroom_tstY = train_test_split(mushroomX, 
                                                                              mushroomY, 
                                                                              test_size=0.3, 
                                                                              random_state=0, 
                                                                              stratify=mushroomY) 
wine_trgX, wine_tstX, wine_trgY, wine_tstY = train_test_split(wineX, 
                                                              wineY, 
                                                              test_size=0.3, 
                                                              random_state=0, 
                                                              stratify=wineY)

clusters =  [2, 5, 10, 15, 20, 25, 30, 35]
dims_mushroom = [2, 5, 10, 15, 20, 30, 40, 50]
dims_wine = [2, 3, 4, 5, 6, 7, 8, 9]

d = wineX.shape[1]
hiddens_wine = hiddens_wine = [(h,)*l for l in [1, 2] for h in [d//4, d//2, d, d*2]]
alphas = [10**-x for x in np.arange(-1,5.01,1/2)]

In [31]:
# PCA
out = './PCA/'
cmap = cm.get_cmap('Spectral') 

np.random.seed(0)

#%% data for 1

pca = PCA(random_state=5)
pca.fit(mushroomX)
tmp = pd.Series(data=pca.explained_variance_) # , index = range(1, 501))
tmp.to_csv(out + 'mushroom_ev.csv')

pca = PCA(random_state=5)
pca.fit(wineX)
tmp = pd.Series(data=pca.explained_variance_) # , index = range(1, 65))
tmp.to_csv(out + 'wine_ev.csv')

metric = 'ev'
problem = 'PCA'

dataset = 'mushroom'
title = 'Explained variance vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

dataset = 'wine'
title = 'Explained variance vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

#%% Data for 2
grid = {'pca__n_components': dims_wine, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
pca = PCA(random_state=5)       
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
pipe = Pipeline([('pca', pca), ('NN', mlp)])
gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

gs.fit(wineX, wineY)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out + 'wine_cv_results.csv')

make_complexity_curve('pca', 'wine', 'n_components', 'PCA')
raise


#%% data for 3
# Set this from chart 2 and dump, use clustering script to finish up
dim = 60
pca = PCA(n_components=dim, random_state=10)
wineX2 = pca.fit_transform(wineX)
wine2 = pd.DataFrame(np.hstack((wineX2, np.atleast_2d(wineY).T)))
cols = list(range(wine2.shape[1]))
cols[-1] = 'Class'
wine2.columns = cols
wine2.to_hdf(out + 'datasets.hdf', 'wine', complib='blosc', complevel=9)

RuntimeError: No active exception to reraise

In [34]:
# ICA
out = './ICA/'

np.random.seed(0)


#raise
#%% data for 1

ica = FastICA(random_state=5)
kurt = {}
for dim in dims_mushroom:
    ica.set_params(n_components=dim)
    tmp = ica.fit_transform(mushroomX)
    tmp = pd.DataFrame(tmp)
    tmp = tmp.kurt(axis=0)
    kurt[dim] = tmp.abs().mean()

kurt = pd.Series(kurt) 
kurt.to_csv(out + 'mushroom_kurt.csv')

ica = FastICA(random_state=5)
kurt = {}
for dim in dims_wine:
    ica.set_params(n_components=dim)
    tmp = ica.fit_transform(wineX)
    tmp = pd.DataFrame(tmp)
    tmp = tmp.kurt(axis=0)
    kurt[dim] = tmp.abs().mean()

kurt = pd.Series(kurt) 
kurt.to_csv(out + 'wine_kurt.csv')

metric = 'kurt'
problem = 'ICA'

dataset = 'mushroom'
title = 'Kurtosis vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

dataset = 'wine'
title = 'Kurtosis vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

#%% Data for 2
grid = {'ica__n_components': dims_wine, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
ica = FastICA(random_state=5)       
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
pipe = Pipeline([('ica', ica), ('NN', mlp)])
gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

gs.fit(wineX, wineY)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out + 'wine_cv_results.csv')
make_complexity_curve('ica', 'wine', 'n_components', 'ICA')
raise

#%% data for 3
# Set this from chart 2 and dump, use clustering script to finish up
dim = 60
ica = FastICA(n_components=dim, random_state=10)
wineX2 = ica.fit_transform(wineX)
wine2 = pd.DataFrame(np.hstack((wineX2, np.atleast_2d(wineY).T)))
cols = list(range(wine2.shape[1]))
cols[-1] = 'Class'
wine2.columns = cols
wine2.to_hdf(out + 'datasets.hdf', 'wine', complib='blosc', complevel=9)

/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


RuntimeError: No active exception to reraise

In [35]:
# Random Forest
out = './RF/'

np.random.seed(0)

#%% data for 1

rfc = RandomForestClassifier(n_estimators=100, 
                             class_weight='balanced', 
                             random_state=5, 
                             n_jobs=-1)
fs_mushroom = rfc.fit(mushroomX, mushroomY).feature_importances_ 
fs_wine = rfc.fit(wineX, wineY).feature_importances_ 

tmp = pd.Series(np.sort(fs_mushroom)[::-1])
tmp.to_csv(out + 'mushroom_fi.csv')

tmp = pd.Series(np.sort(fs_wine)[::-1])
tmp.to_csv(out + 'wine_fi.csv')

metric = 'fi'
problem = 'RF'

dataset = 'mushroom'
title = 'Feature importance vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

dataset = 'wine'
title = 'Feature importance vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

#%% Data for 2
filtr = ImportanceSelect(rfc)
grid = {'filter__n': dims_wine, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}  
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
pipe = Pipeline([('filter', filtr), ('NN', mlp)])
gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

gs.fit(wineX, wineY)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out + 'wine_cv_results.csv')
make_complexity_curve('filter', 'wine', 'n', 'RF')
raise

#%% data for 3
# Set this from chart 2 and dump, use clustering script to finish up
dim = 40
filtr = ImportanceSelect(rfc, dim)
wineX2 = filtr.fit_transform(wineX, wineY)
wine2 = pd.DataFrame(np.hstack((wineX2, np.atleast_2d(wineY).T)))
cols = list(range(wine2.shape[1]))
cols[-1] = 'Class'
wine2.columns = cols
wine2.to_hdf(out +'datasets.hdf', 'wine', complib='blosc', complevel=9)

RuntimeError: No active exception to reraise

In [38]:
# Random Projection

out = './RP/'
cmap = cm.get_cmap('Spectral') 

np.random.seed(0)

#raise
#%% data for 1

tmp = defaultdict(dict)
for i, dim in product(range(8), dims_mushroom):
    rp = SparseRandomProjection(random_state=i, n_components=dim)
    tmp[dim][i] = pairwiseDistCorr(rp.fit_transform(mushroomX), mushroomX)
tmp = pd.DataFrame(tmp).T
tmp.to_csv(out + 'mushroom_cor.csv')

tmp = defaultdict(dict)
for i, dim in product(range(8), dims_wine):
    rp = SparseRandomProjection(random_state=i, n_components=dim)
    tmp[dim][i] = pairwiseDistCorr(rp.fit_transform(wineX), wineX)
tmp = pd.DataFrame(tmp).T
tmp.to_csv(out + 'wine_cor.csv')

metric = 'cor'
title = 'Correlation vs Number of components'
problem = 'RP'

dataset = 'mushroom'
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

dataset = 'wine'
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

tmp = defaultdict(dict)
for i, dim in product(range(8), dims_mushroom):
    rp = SparseRandomProjection(random_state=i, n_components=dim)
    rp.fit(mushroomX)    
    tmp[dim][i] = reconstructionError(rp, mushroomX)
tmp = pd.DataFrame(tmp).T
tmp.to_csv(out + 'mushroom_rec_error.csv')

tmp = defaultdict(dict)
for i, dim in product(range(8), dims_wine):
    rp = SparseRandomProjection(random_state=i, n_components=dim)
    rp.fit(wineX)  
    tmp[dim][i] = reconstructionError(rp, wineX)
tmp =pd.DataFrame(tmp).T
tmp.to_csv(out + 'wine_rec_error.csv')

metric = 'rec_error'
problem = 'RP'

dataset = 'mushroom'
title = 'Reconstruction error vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

dataset = 'wine'
title = 'Reconstruction error vs Number of components - {} - {}'.format(problem, dataset)
run_plot_metric_vs_comp(out, dataset, metric, problem, title)

#%% Data for 2
grid = {'rp__n_components': dims_wine, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
rp = SparseRandomProjection(random_state=5)           
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
pipe = Pipeline([('rp', rp), ('NN', mlp)])
gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

gs.fit(wineX, wineY)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out + 'wine_cv_results.csv')

make_complexity_curve('rp', 'wine', 'n_components', 'RP')
raise

#%% data for 3
# Set this from chart 2 and dump, use clustering script to finish up
dim = 60
rp = SparseRandomProjection(n_components=dim, random_state=5)
wineX2 = rp.fit_transform(wineX)
wine2 = pd.DataFrame(np.hstack((wineX2, np.atleast_2d(wineY).T)))
cols = list(range(wine2.shape[1]))
cols[-1] = 'Class'
wine2.columns = cols
wine2.to_hdf(out + 'datasets.hdf', 'wine', complib='blosc', complevel=9)

RuntimeError: No active exception to reraise

In [40]:
# Benchmark

out = './BASE/'
np.random.seed(0)

#%% benchmarking for chart type 2

grid = {'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
pipe = Pipeline([('NN', mlp)])
gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

gs.fit(wineX, wineY)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out + 'wine_cv_results.csv')
raise

RuntimeError: No active exception to reraise

In [ ]:
# Clustering
# python clustering.py PCA
# python clustering.py BASE
# python clustering.py ICA
# python clustering.py RP
# python clustering.py RF

for dim_red in ['PCA', 'BASE', 'ICA', 'RP', 'RF']:
    out = './{}/'.format(dim_red)

    np.random.seed(0)

    #%% Data for 1-3
    SSE = defaultdict(dict)
    ll = defaultdict(dict)
    acc = defaultdict(lambda: defaultdict(dict))
    adjMI = defaultdict(lambda: defaultdict(dict))
    km = kmeans(random_state=5)
    gmm = GMM(random_state=5)

    st = clock()
    for k in clusters:
        km.set_params(n_clusters=k)
        gmm.set_params(n_components=k)
        km.fit(mushroomX)
        gmm.fit(mushroomX)
        SSE[k]['mushroom'] = km.score(mushroomX)
        ll[k]['mushroom'] = gmm.score(mushroomX)    
        acc[k]['mushroom']['Kmeans'] = cluster_acc(mushroomY, km.predict(mushroomX))
        acc[k]['mushroom']['GMM'] = cluster_acc(mushroomY, gmm.predict(mushroomX))
        adjMI[k]['mushroom']['Kmeans'] = ami(mushroomY, km.predict(mushroomX))
        adjMI[k]['mushroom']['GMM'] = ami(mushroomY, gmm.predict(mushroomX))

        km.fit(wineX)
        gmm.fit(wineX)
        SSE[k]['wine'] = km.score(wineX)
        ll[k]['wine'] = gmm.score(wineX)
        acc[k]['wine']['Kmeans'] = cluster_acc(wineY, km.predict(wineX))
        acc[k]['wine']['GMM'] = cluster_acc(wineY, gmm.predict(wineX))
        adjMI[k]['wine']['Kmeans'] = ami(wineY, km.predict(wineX))
        adjMI[k]['wine']['GMM'] = ami(wineY, gmm.predict(wineX))
        print(k, clock()-st)

    SSE = (-pd.DataFrame(SSE)).T
    SSE.rename(columns=lambda x: x + ' SSE (left)', inplace=True)
    ll = pd.DataFrame(ll).T
    ll.rename(columns=lambda x: x + ' log-likelihood', inplace=True)
    acc = pd.Panel(acc)
    adjMI = pd.Panel(adjMI)

    SSE.to_csv(out + 'SSE.csv')
    ll.to_csv(out + 'loglikelihood.csv')
    acc.ix[:, :, 'mushroom'].to_csv(out + 'mushroom_acc.csv')
    adjMI.ix[:, :, 'mushroom'].to_csv(out + ' mushroom_adjMI.csv')
    acc.ix[:, :, 'wine'].to_csv(out + 'wine_acc.csv')
    adjMI.ix[:, :, 'wine'].to_csv(out + ' wine_adjMI.csv')

    #%% NN fit data (2,3)
    grid = {'km__n_clusters': clusters, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
    mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
    km = kmeans(random_state=5)
    pipe = Pipeline([('km', km), ('NN', mlp)])
    gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

    gs.fit(wineX, wineY)
    tmp = pd.DataFrame(gs.cv_results_)
    tmp.to_csv(out + 'wine_kmeans_cv_results.csv')

    grid = {'gmm__n_components': clusters, 'NN__alpha': alphas, 'NN__hidden_layer_sizes': hiddens_wine}
    mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=5)
    gmm = myGMM(random_state=5)
    pipe = Pipeline([('gmm', gmm), ('NN', mlp)])
    gs = GridSearchCV(pipe, grid, verbose=0, cv=5)

    gs.fit(wineX, wineY)
    tmp = pd.DataFrame(gs.cv_results_)
    tmp.to_csv(out + 'wine_gmm_cv_results.csv')

    # %% For chart 4/5
    mushroomX2D = TSNE(verbose=0, random_state=5).fit_transform(mushroomX)
    wineX2D = TSNE(verbose=0, random_state=5).fit_transform(wineX)

    mushroom2D = pd.DataFrame(np.hstack((mushroomX2D, np.atleast_2d(mushroomY).T)), columns=['x','y','target'])
    wine2D = pd.DataFrame(np.hstack((wineX2D, np.atleast_2d(wineY).T)), columns=['x','y','target'])

    mushroom2D.to_csv(out + 'mushroom2D.csv')
    wine2D.to_csv(out + 'wine2D.csv')

2 2.040509999999813
5 5.629069999999956
10 13.23928799999976
15 22.536873999999443
20 35.670429999999214
25 55.44510300000002
30 75.12658199999987
35 99.03682200000003


/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:53: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:54: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing o

2 2.2995000000009895
5 6.039060999999492
10 13.613205999998172
15 23.46480900000097
20 35.71979400000055
25 54.01454499999818
30 73.12796199999866
35 95.61445900000035


/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:53: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/francopiccolo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:54: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing o

In [ ]:
# madelon tricks
out = './PCA/'
cmap = cm.get_cmap('Spectral') 

np.random.seed(0)

mushroom = pd.read_hdf('./BASE/datasets.hdf','mushroom')        
mushroomX = mushroom.drop('Class',1).copy().values
mushroomY = mushroom['Class'].copy().values
scaler =StandardScaler()

mushroom_test = pd.read_hdf('./BASE/datasets.hdf','mushroom')        
mushroom_tstX = mushroom_test.drop('Class',1).copy().values
mushroom_tstY = mushroom_test['Class'].copy().values
from sklearn.ensemble import RandomForestClassifier

mushroomX = scaler.fit_transform(mushroomX)
mushroom_tstX = scaler.transform(mushroom_tstX)


#Reproduce best estimator so far
#if __name__=='__main__':
#    rfc = RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=5,n_jobs=7)
#    filtr = ImportanceSelect(rfc)
#    grid ={'filter__n':[20],'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
#    mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
#    pipe = Pipeline([('filter',filtr),('NN',mlp)])
#    gs = GridSearchCV(pipe,grid,verbose=10,cv=5)    
#    gs.fit(mushroomX,mushroomY)
#    print('Best CV Score {}'.format(gs.best_score_))
#    print('Test Score {}'.format(gs.score(mushroom_tstX,mushroom_tstY)))
#    rf_features = gs.best_estimator_.steps[0][1].model.feature_importances_.argsort()[::-1][:20]
    
    
# Use PCA to find true correct featuers
pca = PCA(random_state=5, n_components=500)
pca.fit(mushroomX)
ve = pd.Series(pca.explained_variance_)
ve.plot()
plt.xlabel('Component')
plt.ylabel('Variance Explained')
tmp = pd.DataFrame(pca.components_)
tmp=tmp.iloc[-15:,:]
pca_features=tmp.columns[tmp.abs().max()>0.1]

    
xx= mushroomX[:, pca_features]
xx_tst = mushroom_tstX[:, pca_features]

## NN testing - standard param set
#grid ={'alpha':nn_reg,'hidden_layer_sizes':nn_arch}
#mlp = MLPClassifier(activation='relu',max_iter=3000,early_stopping=False,random_state=5)
#gs = GridSearchCV(mlp,param_grid=grid,verbose=10,cv=5)
#gs.fit(mushroomX[:,pca_features],mushroomY)
#print('NN - Standard params - Best CV Score {}'.format(gs.best_score_))
#print('NN - Standard params - Test Score {}'.format(gs.score(xx_tst,mushroom_tstY)))
#
#
#
## NN testing - standard param set
#grid ={'alpha':[1e-4,1e-5,1e-6],'hidden_layer_sizes':[(200,100,100,64,100,100,200)]}
#mlp = MLPClassifier(activation='relu',max_iter=3000,early_stopping=False,random_state=5)
#gs = GridSearchCV(mlp,param_grid=grid,verbose=10,cv=5)
#gs.fit(mushroomX[:,pca_features],mushroomY)
#print('NN - Big network- Best CV Score {}'.format(gs.best_score_))
#print('NN - Big network - Test Score {}'.format(gs.score(xx_tst,mushroom_tstY)))


#KNN
knn = KNeighborsClassifier()
grid = {'n_neighbors':range(1, 25, 1),'p':[1, 2], 'weights': ['uniform', 'distance']}
gs = GridSearchCV(knn, param_grid=grid, cv=5, verbose=0)
gs.fit(xx,mushroomY)
print('KNN - Best CV Score {}'.format(gs.best_score_))
print('KNN - Test Score {}'.format(gs.score(xx_tst, mushroom_tstY)))


# SVM
dis = pairwise_distances(xx)
m = np.median(dis)
gammas = [(1/m)*x for x in np.arange(0.1,2.1,0.1)]+[0.1,0.2,0.3,0.4,0.5]
gammas = np.arange(0.1,0.9,0.05)

gammas = [(1/m)*x for x in np.arange(0.1,2.1,0.1)]
param_grid={'gamma':gammas,'C':[10**x for x in [-1,0,1,2,3]]}
gs = GridSearchCV(SVC(kernel='rbf', C=1), param_grid=param_grid, cv=5, verbose=10, n_jobs=1)
gs.fit(xx, mushroomY)
print('SVM - Best CV Score {}'.format(gs.best_score_))
print('SVM - Test Score {}'.format(gs.score(xx_tst, mushroom_tstY)))